This Code Will Be Used To Create 6 Different Features For A Given Text


In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

# Load model
sarcasticModel = load_model('sarcasmModel.h5')
sentimentModel = load_model('sentimentModel.h5')


In [120]:
#Creating Tokenizers
#Sarcasm
sarcasmDF = pd.read_csv("Clean_Sarcasm_Reddit.csv")
for idx,row in sarcasmDF.iterrows():
    row[1] = row[1].replace('rt',' ')
    
max_words = 2000
sarcasticTokenizer = Tokenizer(num_words=max_words, split=' ')
sarcasticTokenizer.fit_on_texts(sarcasmDF['headline'].values)
#Sentiment 
sentimentDF = pd.read_csv("Twitter_Data_2.csv")
max_words = 5000
max_len=50

sentimentTokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')

sentimentTokenizer.fit_on_texts(sentimentDF['clean_text'])

In [100]:
def predictSentiment(text):
  '''Function to predict sentiment class of the passed text'''
    
  sentimentClasses = ['Negative', 'Neutral', 'Positive']
  max_len=50
    
  # Transforms text to a sequence of integers using a tokenizer object
  xt = sentimentTokenizer.texts_to_sequences(text)
  # Pad sequences to the same length
  xt = pad_sequences(xt, padding='post', maxlen=max_len)
  # Do the prediction using the loaded model
  yt = sentimentModel.predict(xt).argmax(axis=1)
  # Print the predicted sentiment

  return sentimentClasses[yt[0]]  

In [121]:


def predictSarcasm(text):
  '''Function to predict sarcasm class of the passed text'''
    
  sarcasticClasses = ['Not-Sarcastic', 'Sarcastic']
  max_len=50
    
  # Transforms text to a sequence of integers using a tokenizer object
  xt = sarcasticTokenizer.texts_to_sequences(text)
  print(xt)
  # Pad sequences to the same length
  xt = pad_sequences(xt, padding='post', maxlen=max_len)
  # Do the prediction using the loaded model
  print(xt)
  yt = sarcasticModel.predict(xt).argmax(axis=1)
  # Print the predicted sentiment
  return sarcasticClasses[yt[0]] 

In [106]:
!pip install text2emotion
import text2emotion as te

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 133kB 6.2MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [107]:
def convertTextToEmotions(text):
    return te.get_emotion(text)
    

In [108]:
!pip install textstat

     |████████████████████████████████| 102kB 3.6MB/s 
     |████████████████████████████████| 1.9MB 7.4MB/s 


In [109]:
import textstat as ts

In [110]:
def convertTextToStats(text):
  # get the text for the row and create the scores 
  readingEase = ts.flesch_reading_ease(text)
  dcScore = ts.dale_chall_readability_score(text)
  lFormula = ts.linsear_write_formula(text)
  return {"Reading Ease": readingEase, "Dale-Chall": dcScore, "Linsear Write": lFormula}

In [134]:
def extractTextFeatures(text):
  # Emotions dictionary
  emotions = convertTextToEmotions(text)
  # Stat dicitonary
  stats = convertTextToStats(text)
  # ML dictinoary 
  ml = {"Sarcasm Indicator": predictSarcasm([text]), "Sentiment": predictSentiment([text])} 
  return {**emotions,  **stats, **ml}

In [135]:
extractTextFeatures("I want to go on a walk today with my friends")

[[65, 176, 1, 178, 7, 6, 1258, 477, 9, 78, 218]]
[[  65  176    1  178    7    6 1258  477    9   78  218    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]]


{'Angry': 0.0,
 'Dale-Chall': 0.55,
 'Fear': 0.0,
 'Happy': 1.0,
 'Linsear Write': 4.5,
 'Reading Ease': 102.61,
 'Sad': 0.0,
 'Sarcasm Indicator': 'Sarcastic',
 'Sentiment': 'Neutral',
 'Surprise': 0.0}